# MMVC_Trainerの設定ファイルの作成
ver.2022/12/30

「Google Colaboratory」を利用したMMVCの学習に必要なファイルを生成します。


​

In [1]:
#@title ## 0 ノートブックの準備

#@markdown このノートブックのセットアップを行います。セルを実行し、完了したら次に進んでください。

#debug用ディレクトリの作成
!rm -rf /mmvc-debug
!mkdir /mmvc-debug

#現在時刻の取得
import datetime
ｊst_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(jst_delta, 'JST')
now = datetime.datetime.now(JST)
nowt = now.strftime('%Y%m%d%H%M%S')

#python管理
import sys

#出力記録用カスタムマジック %%ccapture
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic
from IPython.utils.capture import capture_output

@magic_arguments.magic_arguments()
@magic_arguments.argument('output', type=str, default='', nargs='?')

@register_cell_magic
def ccapture(line, cell):
    args = magic_arguments.parse_argstring(ccapture, line)
    with capture_output() as outputs:
        get_ipython().run_cell(cell)
    if args.output:
        get_ipython().user_ns[args.output] = outputs
    
    outputs()

​

In [2]:
%%ccapture one_mount_gdrive
print("----------------------------------------------------------------------------------------------------")
print("1 Google Driveをマウント")

#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

----------------------------------------------------------------------------------------------------
1 Google Driveをマウント
Mounted at /content/drive


​

In [3]:
%%ccapture two_cd_mmvc_trainer
print("----------------------------------------------------------------------------------------------------")
print("2 MMVC_Trainerディレクトリに移動")

#@title ## 2 MMVC_Trainerディレクトリに移動
#@markdown ​マウントしたGoogle DriveのMMVC_Trainerディレクトリに移動します。

#@markdown Google DriveでMMVC_Trainerの場所を確認し、以下でパスを指定してください。

#@markdown 正しいパスが指定されていれば、以下のようなメッセージが表示されます。

#@markdown ```
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown ```
#@markdown


#@markdown ​
#@markdown ### Settings
directory = "/content/drive/MyDrive/MMVC_Trainer" #@param {type:"string"}

%cd $directory
!ls -1

----------------------------------------------------------------------------------------------------
2 MMVC_Trainerディレクトリに移動
/content/drive/MyDrive/MMVC_Trainer
attentions.py
commons.py
configs
create_dataset_jtalk.py
dataset
data_utils.py
docs
edit_iteration.py
filelists
fine_model
LICENSE
losses.py
mel_processing.py
models.py
modules.py
monotonic_align
notebook
onnx_bench.py
onnx_export.py
preprocess.py
README.md
requirements.txt
requirements_wsl2.txt
text
train_ms.py
train.py
transforms.py
utils.py
version.txt


​

In [4]:
%%ccapture three_install_library
print("----------------------------------------------------------------------------------------------------")
print("3 ライブラリのインストール")

#@title ## 3 ライブラリのインストール
#@markdown 数分かかります。

#@markdown 成功すると、以下のようなメッセージが表示されます。

#@markdown ```
#@markdown　Successfully installed pyopenjtalk-x.x.x (x.x.xはバージョン名)
#@markdown ```

!pip install pyopenjtalk

----------------------------------------------------------------------------------------------------
3 ライブラリのインストール
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 24.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyopenjtalk: filename=pyopenjtalk-0.3.0-cp38-cp38-linux_x86_64.whl size=4477684 sha256=440df8d1c87876813ad763177cba836bdd789bdf4400ee22180dabe54608c058
  Stored in directory: /root/.cache/pip/wheels/88/74/65/3fffc9c6c6a4b625cd5108d4e42e599d6aee5fcf02b1480119
Successfully built pyopenjtalk


​

In [7]:
%%ccapture four_create_config
print("----------------------------------------------------------------------------------------------------")
print("4 config系Fileを作成する")

#@title ## 4 config系Fileを作成する
#@markdown 学習に必要なファイルを実行します。

#@markdown 必要に応じて以下のSettingsを変更してから、実行してください。

#@markdown 実行時に  
#@markdown　**「WARNING: JPCommonLabel_insert_pause() in jpcommon_label.c: First mora should not be short pause.」**  
#@markdown と表示されますが、無視して大丈夫です。 


#@markdown ​
#@markdown ### Settings
#@markdown -f：作成するconfig系Fileの名前です。  よく分からない場合は変更不要です。
config_name = "train_config" #@param {type:"string"}

#@markdown -s：サンプリングレートです。datasetの音声のサンプリングレートに合わせて指定ください。よく分からない場合は変更不要です。  
sample_rate = 24000 #@param {type:"integer"}

#@markdown -t：ターゲットとなるキャラクターの設定です。
#@markdown ずんだもんの学習を行いたい場合は「100」 、
#@markdown 九州そらの学習を行いたい場合は「101」、
#@markdown 四国めたんの学習を行いたい場合は「102」、  
#@markdown 春日部つむぎの学習を行いたい場合は「103」
#@markdown を設定するとクオリティがあがる**かも**しれません。
#@markdown 非サポートキャラは「108」を指定します。
character_select = 100 #@param [100, 101, 102, 103, 108]

#@markdown -m：複数話者の学習を同時に行いたい場合に使用します。
#@markdown multi_speaker_correspondence.txt を ```textful配下のディレクトリ名|0～109の番号``` となるように書き換えた後、オンにしてください。このオプションがtrueの場合、-tは無視されます。
multi_speakers = False #@param {type:"boolean"}

#@markdown -cについては、普通に使用する場合には必要ありません。

if multi_speakers == 'True':
  !python create_dataset_jtalk.py -f $config_name -s $sample_rate -m dataset/multi_speaker_correspondence.txt
else:
  !python create_dataset_jtalk.py -f $config_name -s $sample_rate -t $character_select

----------------------------------------------------------------------------------------------------
4 config系Fileを作成する
train_config
['えっうそでしょ。', '']
Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"
dic.tar.gz: 100% 22.6M/22.6M [00:00<00:00, 63.8MB/s]
Extracting tar file /usr/local/lib/python3.8/dist-packages/pyopenjtalk/dic.tar.gz
sil-e-cl-u-s-o-d-e-sh-o-sil
dataset/textful/00_myvoice/wav/emotion001.wav|107|sil-e-cl-u-s-o-d-e-sh-o-sil
['しゅゔぁいつぁーわみならうべきにんげんです。', '']
sil-sh-u-v-a-i-ts-a-a-w-a-m-i-n-a-r-a-u-b-e-k-i-n-i-N-g-e-N-d-e-s-U-sil
dataset/textful/00_myvoice/wav/emotion002.wav|107|sil-sh-u-v-a-i-ts-a-a-w-a-m-i-n-a-r-a-u-b-e-k-i-n-i-N-g-e-N-d-e-s-U-sil
['でーゔぃすさんわとてもつかれているよーにみえる。', '']
sil-d-e-e-v-i-s-u-s-a-N-w-a-t-o-t-e-m-o-ts-U-k-a-r-e-t-e-i-r-u-y-o-o-n-i-m-i-e-r-u-sil
dataset/textful/00_myvoice/wav/emotion003.wav|107|sil-d-e-e-v-i-s-u-s-a-N-w-a-t-o-t-e-m-o-ts-U-k-a-r-e-t-e-i-r-u-y-o-o-n-i-m-i-e-r-u-sil
['すてぃーゔわじぇーんからてがみを

​

In [8]:
%%ccapture five_check_files
print("----------------------------------------------------------------------------------------------------")
print("5 確認")

#@title ## 5 確認
#@markdown 以下のようなメッセージが表示されれば、保存が正常に完了しています。
#@markdown ```
#@markdown Directory: filelists
#@markdown 指定したconfig系Fileの名前_textful.txt
#@markdown 指定したconfig系Fileの名前_textful_val.txt
#@markdown 指定したconfig系Fileの名前_textless.txt
#@markdown 指定したconfig系Fileの名前_val_textless.txt
#@markdown 指定したconfig系Fileの名前_Correspondence.txt
#@markdown
#@markdown Directory: configs
#@markdown baseconfig.json
#@markdown 指定したconfig系Fileの名前.json
#@markdown ```

print("Directory: filelists")
!ls -1 {directory}/filelists
print("\n")
print("Directory: configs")
!ls -1 {directory}/configs

----------------------------------------------------------------------------------------------------
5 確認
Directory: filelists
train_config_Correspondence.txt
train_config_textful.txt
train_config_textful_val.txt
train_config_textless.txt
train_config_val_textless.txt


Directory: configs
baseconfig.json
train_config.json


​

## サポート専用

**以下はお問い合わせの際、指示があった場合のみ使用してください。**

In [ ]:
#@markdown **このセルは無視してください。**

#@markdown このセルは、セルが一括で実行されることを防ぐためのものです。

#@markdown  実行してしまった場合は、左側のアイコンをクリックしてセルを終了してください。

#一括実行の阻止
import time
time.sleep(86400)

​

In [ ]:
#@title ## サポート用ファイルの作成

#@markdown セルを実行すると内部処理が行われ、zipファイルが操作中のPC(またはタブレットなど)にダウンロードされます。

#@markdown ダウンロードされるzipファイルには、以下のファイルや情報が含まれます。

#@markdown * MMVC_Trainerフォルダ内のファイルの一覧
#@markdown * このノートブックで使用されている変数(自動的に設定されるものと、ユーザーが入力するものがあります)のリスト
#@markdown * このセッション内の出力
#@markdown * configsフォルダ内及びlogsフォルダ内のファイル

#@markdown これらには、ユーザーの個人情報が含まれる可能性があります。ダウンロード完了後、ファイルを共有する前に、必ず内容をご確認ください。

#@markdown ファイルのダウンロードが完了したら、ランタイムを切断してください。

#ファイルの準備
variable_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-variable.txt"
tree_dic_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-tree_dic.txt"
export_zip = "/mmvc-debug-" + str(nowt)
export_zipp = "/mmvc-debug-" + str(nowt) + ".zip"

#変数の値を保存
#whos使うと長い文字列が省略されるため、変数毎に取得する
##変数の一覧を取得
vlist = %who_ls
##それぞれの変数で値を取得してファイルに保存
with open(variable_txt, 'w') as f:
  for ev in vlist:
    try:
      #変数名(str)
      print(ev, end=' : ', file=f) 
      #変数の型(変数名がstrとなっているためevalでkeyに直す)
      print(type(eval(ev)), end=' : ', file=f)
      #変数の内容(変数名がstrとなっているためevalでkeyに直す)
      print(eval(ev), file=f)
    except:
      pass

#tree
!apt install tree
##ディレクトリ内以下
import traceback
with open(tree_dic_txt, 'w') as f:
  try:
    !tree {directory} > {tree_dic_txt}
  except Exception as e:
    print("An error occurred!", file=f)
    print(e, file=f)
    print(traceback.print_exc(), file=f)

#logsとconfigsの保存
#directoryが無かったらmkdirしてno-dic.txt置く
!if [ -d {directory}/logs ]; then if [ -z "$(ls {directory}/logs)" ]; then touch {directory}/logs/no-file.txt;else cp -rp {directory}/logs /mmvc-debug/logs; fi;else mkdir /mmvc-debug/logs && touch /mmvc-debug/logs/no-dic.txt; fi
!if [ -d {directory}/configs ]; then if [ -z "$(ls {directory}/configs)" ]; then touch {directory}/configs/no-file.txt;else cp -rp {directory}/configs /mmvc-debug/configs; fi;else mkdir /mmvc-debug/configs && touch /mmvc-debug/configs/no-dic.txt; fi

#直近のtracebackの保存
with open('/mmvc-debug/traceback.txt', 'w') as f:
  try:
    print(sys.last_type, sys.last_value, sys.last_traceback, file=f)
  except:
    pass

#ccaptureの保存
!mkdir /mmvc-debug/ccapture
with open('/mmvc-debug/ccapture/one_mount_gdrive.txt', 'w') as f:
  try:
    print(one_mount_gdrive, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/two_cd_mmvc_trainer.txt', 'w') as f:
  try:
    print(two_cd_mmvc_trainer, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/three_install_library.txt', 'w') as f:
  try:
    print(three_install_library, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/four_create_config.txt', 'w') as f:
  try:
    print(four_create_config, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/five_check_files.txt', 'w') as f:
  try:
    print(five_check_files, file=f)
  except:
    pass

#zipにまとめる
!apt install zip
!zip {export_zip} -r /mmvc-debug

#colabのfilesモジュールを使ってダウンロード
from google.colab import files
files.download(export_zipp)